In [55]:
import os
import time

import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam    # 가중치, bias 최적화

# csv 파일에서 불러오기
r_cols = ['memberno', 'productno', 'rating'] # 사용자 아이디, 아이템 아이디, 평점 데이터
ratings = pd.read_csv('u.data', names=r_cols,  sep=',', encoding='utf-8')
ratings = ratings[['memberno', 'productno', 'rating']].astype(int)            # timestamp 제거

# train test 분리
from sklearn.utils import shuffle

TRAIN_SIZE = 0.75
ratings = shuffle(ratings) # DataFrame형태의 데이터를 무작위로 섞음
cutoff = int(TRAIN_SIZE * len(ratings))
print('cutoff:', cutoff)
ratings_train = ratings.iloc[:cutoff] # 0:75000b
ratings_test = ratings.iloc[cutoff:]  # 75000:100000
mu = ratings_train.rating.mean() 

cutoff: 225


In [56]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adamax
from tensorflow.keras.callbacks import EarlyStopping   # 학습 자동 중지
from tensorflow.keras.callbacks import ModelCheckpoint # 우수한 학습 모델 파일 저장

In [57]:
model1 = load_model('./personalization_mu_mlp.h5')
model2 = load_model('./personalization_mu_mlp2.h5')

In [58]:
# 테스트 데이터 Prediction
user_ids = ratings_test.memberno.values[0:]
product_ids = ratings_test.productno.values[0:]
predictions = model1.predict([user_ids, product_ids]) + mu
#print("Actuals: \n", ratings_test[0:])
#print( )
#print("Predictions: \n", predictions)

3/3 [==============================] - 0s 5ms/step


In [59]:
df = ratings_test[0:]
df

,memberno,productno,rating
201,49,11,4
62,24,8,4
260,3,16,3
142,4,17,2
225,15,11,3
...,...,...,...
218,10,7,5
32,6,11,4
207,9,15,3
179,13,8,3


In [60]:
df['predictions'] = predictions
df

C:\Users\user\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,memberno,productno,rating,predictions
201,49,11,4,2.972880
62,24,8,4,4.649117
260,3,16,3,3.051898
142,4,17,2,3.817217
225,15,11,3,3.291678
...,...,...,...,...
218,10,7,5,4.297581
32,6,11,4,3.653126
207,9,15,3,4.140430
179,13,8,3,3.690742


In [61]:
df['deference']=df['rating']-df['predictions']

C:\Users\user\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
df

,memberno,productno,rating,predictions,deference
201,49,11,4,2.972880,1.027120
62,24,8,4,4.649117,-0.649117
260,3,16,3,3.051898,-0.051898
142,4,17,2,3.817217,-1.817217
225,15,11,3,3.291678,-0.291678
...,...,...,...,...,...
218,10,7,5,4.297581,0.702419
32,6,11,4,3.653126,0.346874
207,9,15,3,4.140430,-1.140430
179,13,8,3,3.690742,-0.690742


In [63]:
abs(df['deference']).sum()

76.5859305858612

In [47]:
# 테스트 데이터 Prediction
user_ids = ratings_test.memberno.values[0:]
product_ids = ratings_test.productno.values[0:]
predictions = model2.predict([user_ids, product_ids]) + mu
#print("Actuals: \n", ratings_test[0:])
#print( )
#print("Predictions: \n", predictions)

3/3 [==============================] - 0s 5ms/step


In [48]:
df = ratings_test[0:]
df

,memberno,productno,rating
76,7,10,4
45,40,11,5
254,1,15,3
155,30,19,4
289,23,5,5
...,...,...,...
229,47,20,5
255,49,11,4
99,43,11,4
117,45,25,3


In [49]:
df['predictions'] = predictions
df

C:\Users\user\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,memberno,productno,rating,predictions
76,7,10,4,2.520144
45,40,11,5,5.332174
254,1,15,3,3.600221
155,30,19,4,2.433885
289,23,5,5,4.850034
...,...,...,...,...
229,47,20,5,3.739044
255,49,11,4,4.111698
99,43,11,4,4.297853
117,45,25,3,2.474763


In [50]:
df['deference']=df['rating']-df['predictions']

C:\Users\user\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
df

,memberno,productno,rating,predictions,deference
76,7,10,4,2.520144,1.479856
45,40,11,5,5.332174,-0.332174
254,1,15,3,3.600221,-0.600221
155,30,19,4,2.433885,1.566115
289,23,5,5,4.850034,0.149966
...,...,...,...,...,...
229,47,20,5,3.739044,1.260956
255,49,11,4,4.111698,-0.111698
99,43,11,4,4.297853,-0.297853
117,45,25,3,2.474763,0.525237


In [54]:
abs(df['deference']).sum()

64.84194922447205